In [38]:
import os
import time
import copy
import glob
import cv2
import shutil

import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [39]:
%pwd

'c:\\Dev\\RPA_proj_local\\Img_similarity_model'

In [40]:
# 이미지 데이터 전처리 방법 정의

data_path = '../dataset/resnet_dataset/train'

trsf = transforms.Compose(
    [
        transforms.Resize([256, 256]),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]
)
# torchvision.transforms -> 이미지 데이터를 변환하여 모델의 입력으로 사용할 수 있게 변환
# Resize : 이미지의 크기를 재조정
# RandomResizedCrop : 랜덤한 크기 및 비율로 자름
# RandomHorizontalFlip : 이미지를 랜덤하게 수평으로 뒤집음
# ToTensor : 이미지 데이터를 텐서로 변환

train_dataset = torchvision.datasets.ImageFolder(
    data_path, transform= trsf
)
# ImageFolder -> 데이터로더가 데이터를 불러올 경로와 방법

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size= 32,
    num_workers= 8,
    shuffle= True
)

print(len(train_dataset))

63


In [41]:
# 사전 훈련된 모델
resnet50 = models.resnet50(pretrained= True)

# 파라미터 학습 유무 지정
def set_parameter_requires_grad(model, feature_extracting= True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

set_parameter_requires_grad(resnet18)

# 완전연결층 추가
resnet18.fc = nn.Linear(512, 2)

# 파라미터 값 확인
for name, param in resnet18.named_parameters():
    if param.requires_grad:
        print(name, param.data)

c:\Users\Ino\anaconda3\envs\RPA_proj\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Ino\anaconda3\envs\RPA_proj\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


fc.weight tensor([[-0.0115,  0.0221, -0.0308,  ...,  0.0277, -0.0276,  0.0280],
        [-0.0123,  0.0331,  0.0132,  ...,  0.0334, -0.0002, -0.0437]])
fc.bias tensor([ 0.0277, -0.0405])


In [42]:
# 모델 객체 생성, 손실 함수 정의

model = models.resnet50(pretrained= True)

for param in model.parameters(): # 합성곱층 가중치 고정
    param.required_grad = False

model.fc = torch.nn.Linear(2048, 3)
for param in model.fc.parameters(): # 완전연결층은 학습
    param.requires_grad = True

opt = torch.optim.Adam(model.fc.parameters())
cost = torch.nn.CrossEntropyLoss()

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [43]:
p = model.parameters()
print(type(model.parameters()))
print(p)

<class 'generator'>
<generator object Module.parameters at 0x000001F35249E880>


In [44]:
# 모델 학습 함수

def train_model(model, dataloaders, criterion, optmizer, device, num_epochs= 13, is_train= True):
    since = time.time()
    acc_history = []
    loss_history = []
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs-1))
        print('-'*10)

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            model.to(device)
            opt.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            loss.backward()
            opt.step()

            running_loss += loss.item()*inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)

        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        if epoch_acc > best_acc:
            best_acc = epoch_acc

        acc_history.append(epoch_acc.item())
        loss_history.append(epoch_loss)
        torch.save(model.state_dict(), os.path.join('data/catanddog/', '{0:0=2d}.pth'.format(epoch)))
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:0f}m {:0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Acc: {:4f}'.format(best_acc))

    return acc_history, loss_history

In [45]:
# 학습 결과를 옵티마이저에 전달

params_to_update = []

for name, param in resnet18.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print('\t', name)

opt = optim.Adam(params_to_update)

	 fc.weight
	 fc.bias


In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
train_acc_hist, train_loss_hist = train_model(resnet50, train_loader, criterion, opt, device)

Epoch 0/12
----------
Loss: 9.4938 Acc: 0.0000


RuntimeError: Parent directory data/catanddog does not exist.